# Setting of stack phase 

At this phase we are going to set the stacked-phase dataset 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from utils.trainFoldB import loadColumnsCategorical

In [ ]:
import sys
sys.path.append('../dsbase/src/main')
from AdaBoostClassificationDSBase import AdaBoostClassificationDSBaseModel

## Loading the original stacked dataset and shuffle it

In [ ]:
df = pd.read_csv('datasets/train_stack.csv')

In [ ]:
df_frac = df.sample(frac=0.0005)

In [ ]:
df_frac.shape

## Defining the Fold X processing 

In [ ]:
def getColumnFoldX(df, fold_id):
    # Convert every element to a one-elenet List
    print('   dataframe to list ...')
    df_w = df.drop(['MachineIdentifier','HasDetections','fold'], axis=1)
    columns_categorical = df_w.select_dtypes(include=['object']).columns

    # Comluns transformation
    df_end = loadColumnsCategorical(fold_id, df_w, columns_categorical)
    
    # --------------------------------------
    # Load the i-th model and process
    print('   loading model ...')    
    model = AdaBoostClassificationDSBaseModel('AB2',None,None,None,None,None,None)
    model.load('models/fold' + str(fold_id))
    
    print('   Calculating: normalization ...')    
    pre_result = model.scalerX.transform(df_end.values)
    print('   Calculating: probabilities ...')    
    result = model.model.predict_proba(pre_result)
    
    # Set the result as a one-column DataFrame
    print('   Creating result dataset ...')        
    columns_name = [str('f' + str(fold_id))]
    df_result = pd.DataFrame(result[:,1])
    df_result.columns = columns_name
    return df_result

## Lets obtain the final stacked dataset 

In [ ]:
N = 9 # Number of folds
df_stack_set = df_frac.reset_index(drop=True)
for i in range(9):
    print('processing fold ' + str(i+1) + " ...")
    c = getColumnFoldX(df_frac, i+1)
    df_stack_set = df_stack_set.join(c)